In [1]:
# 手写数字识别

# 一、构建CNN（卷积神经网络）-- convolutional neural network

In [2]:
# 1.1. instantiating ）a small convnet
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))

Using TensorFlow backend.


In [3]:
# 1.2 Displaying a summary of the model so far
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [4]:
# 1.3 Addingg a calssifier on top of the convnet
model.add(layers.Flatten()) # 上面模型输出的是3D,所以我们需要铺平为1D
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [5]:
# 1.4 Displaying a summary of the full model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

# 二、训练模型

In [10]:
# 2.1 Training our convnet on MNIST images
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels) , (test_images, test_labels) = mnist.load_data(path = '/home/sally/Sally/DataSets/mnist2.npz')

train_images.shape

(60000, 28, 28)

In [11]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255   # 为什么除255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size= 64)

Epoch 1/5
60000/60000 [==============================] - 131s 2ms/step - loss: 0.1637 - accuracy: 0.9485
Epoch 2/5
60000/60000 [==============================] - 132s 2ms/step - loss: 0.0469 - accuracy: 0.9855
Epoch 3/5
60000/60000 [==============================] - 129s 2ms/step - loss: 0.0311 - accuracy: 0.9902
Epoch 4/5
60000/60000 [==============================] - 129s 2ms/step - loss: 0.0237 - accuracy: 0.9921
Epoch 5/5
60000/60000 [==============================] - 141s 2ms/step - loss: 0.0192 - accuracy: 0.9941


# 三、评估准确率

In [12]:
# 3.1 Evaluating the trained model
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 9s 915us/step


0.9887999892234802

# 四、修改模型

In [14]:
# 4.1 A convnet without pooling layers
model_no_max_pool = models.Sequential()
model_no_max_pool.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1)))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation = 'relu'))

In [15]:
model_no_max_pool.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 22, 22, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [13]:
# 结论：没有最大池化层会存在两个问题
# 1. 学习不到空间层级结构的特征
# 2. 每一层需要处理的参数太多